In [1]:
import requests
import json
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import datetime
import os
from pathlib import Path


In [2]:
userBot = "gpmbot"
pwdBot = "9ALjKza5GmXqxMKNf"

In [3]:
rootUrl = "https://darkroom.global-precious-metals.com"

In [4]:
def drkrmLogin(user=userBot,password=pwdBot, rootUrl = rootUrl):
    """
    login tryout
    """
    apiLogin = "/api/v1/login"
    data = {"user" : user, "password" : password}
    r = requests.post(rootUrl+apiLogin,data=json.dumps(
    data), headers={'Content-Type': 'application/json'})
    if r.json()['status'] == 'success':
        return r.json()['data']['userId'], r.json()['data']['authToken'] 
    else:
        print(r.json())
        raise Exception("Something went wrong")

In [5]:
def post_message(msg, channel="#general", user=userBot,password=pwdBot, rootUrl = rootUrl):
    """
    we authenticate, then post
    """
    apiMsg = "/api/v1/chat.postMessage"
    userId, authToken = drkrmLogin(user=user, password=password, rootUrl=rootUrl)
    payload = { "channel" : channel, "text" : msg}
    response = requests.post(rootUrl+apiMsg,data=json.dumps(
    payload), headers={'Content-Type': 'application/json', 'X-Auth-Token' : authToken, 'X-USer-Id' : userId})
    return response

In [7]:
DCurl = "https://www.dukascopy.com/swiss/english/home/?utm_source=freeserv"

In [8]:
def validatePath(path):
    """
    Checks if path refers to a valid directory. If not an exception is raised.
    """
    
    try:
        os.path.isfile(filename)
    except:
        raise NotADirectoryError("The specified path does not exist or is invalid. The data will be collected and stored in the current working directory in the file: ")

In [9]:
def getDukasTime(soup):
    """
    Returns the extracted time as given on the DukasCopy webpage from where the DukasCopy data is being extracted from.
    
    Parameters:
    
    soup (soup): The BeautifulSoup object containing the parsed HTML representation of the DukasCopy webpage.
    
    
    """
    tdata = soup.find_all("span", {"id": "timeUpdate"})
    dtxt = tdata[0].text
    
    date_time_obj = datetime.datetime.strptime(dtxt, ' %a, %d %b %Y %H:%M:%S GMT')
    
    return date_time_obj

In [10]:
def updateData(NewFrame, filename):
    """
    Updates the pickle file specified bt "filepath" by appending new data from the "NewFrame" dataframe to it.
    
    Parameters:
    
    NewFrame (DataFrame): The dataframe which is to be appended to the end of the pickle file to update the pickle file with the new data.
    filepath (str): The filepath of the pickle file where new dataframe data is to be appended.
    
    """
    if (not filename.endswith(".pkl")):
        raise Exception("The filename specified must end with .pkl extension. The filename provided was: " + filename)
    
    if not os.path.isfile(filename):
        pd.to_pickle(NewFrame, filename)
        
    else:   
        df = pd.read_pickle(filename)
        df = df.append(NewFrame, sort=False)
        df.to_pickle(filename)

In [57]:
def getDukasCopyData(pathToChromeDriver):
    """
    Returns a DataFrame object containing the most recent Gold Data from the DukasCopy website.
    Scrapes the data from the DukasCopy website before formatting and returning the data as a pandas DataFrame.
    
    Parameters:
    pathToChromeDriver (str): Path to the chromedriver executable to be used to open web pages.
    
    Returns:
    DataFrame: A pandas DataFrame containing the most recent Gold Data from the DukasCopy website.
    """
    
    browser = webdriver.Chrome(pathToChromeDriver)
    browser.get(DCurl)
    
    DChtml = browser.page_source
    soup = BeautifulSoup(DChtml, "lxml")
    
    # Finding the exact table with all the required data
    data = soup.find_all("table", {"id": "list"})
    
    # Creating and Modifying dataframe with the table 
    DCDataFrame = pd.read_html(str(data))[0]
    
    # Add a timestamp to the data
    DCDataFrame["timestamp"] = getDukasTime(soup)
    
    # Filter dataframe only to Gold Data
    DCDataFrame = DCDataFrame[DCDataFrame["Live"] == "XAU/USD"]
    
    browser.quit()
    return DCDataFrame

In [12]:
def fetchAndSaveDukas(filename, chromedriverpath):
    """
    Fetches the most recent Gold Data from the DukasCopy website and updates the pickle file specified by "dukasFilepath" with the newest data.
    If no pickle file with the specified name exists, a new one will be created.
    If no filepath is specified, the current working directory will be used and data will be stored in the pickle file .
    
    Parameters:
    dukasFilepath(str): The filepath to the pickle file where the fetched data will be stored.
    chromedriverpath (str): Path to the chromedriver executable to be used to open web pages.
    """
    df = getDukasCopyData(chromedriverpath)
    updateData(df, filename)

In [13]:
def mainDriver(dukaspath="DukasGoldData.pkl", metalsApipath="MetalsAPIGoldData.pkl", chromedriverpath="chromedriver.exe"):
    """
    Drives the entire code by calling the relevant methods and retrieves and stores the gold prices from the sources.
    
    Parameters:
    dukaspath (str): Path to pickle file where you want to store the DukasCopy scraped data. If the pickle file with the specified name does not exist, it will be created in the specified directory.
    metalsApipath (str): Path to pickle file where you want to store the DukasCopy scraped data. If the pickle file with the specified name does not exist, it will be created in the specified directory.
    chromedriverpath (str): Path to the chromedriver executable to be used to open web pages.
    """
    if dukaspath == metalsApipath:
        raise Exception("The parameters dukaspath and metalsApipath cannot be the same.")
    
    try:
        validatePath(dukaspath)
    except NotADirectoryError as e:
        dukaspath = "DukasGoldData.pkl"
        print(repr(e) + dukaspath)
    
    for i in range(5):
        try:
            fetchAndSaveDukas(dukaspath, chromedriverpath)
        except Exception as e:
            print(repr(e))

In [62]:
def getNormalTime(t):
    lst = t.split("T")
    return lst[1] + " " + lst[0]

In [61]:
def sendPrices():
    currData = getDukasCopyData("chromedriver.exe")
    
    try:
        timestamp = currData.iloc[0]["timestamp"]
        bidPrice = currData.iloc[0]["Bid"]
        askPrice = currData.iloc[0]["Ask"]
    except:
        print("Skipped here")
    
    formattedMsg = "GOLD PRICE as of " + str(timestamp) + "\n \n" + "Bid Price: " + str(bidPrice) + "\nAsk Price: " + str(askPrice)
    
    print(timestamp)
    print(currData)
    
    post_message(formattedMsg)

In [64]:
def driver():
    while True:
        sendPrices()
        time.sleep(7200)

In [ ]:
driver()

2020-01-09 09:41:23
      Live      Bid      Ask  Spread           timestamp
7  XAU/USD  1546.73  1547.01    27.7 2020-01-09 09:41:23


In [ ]:
toto = datetime.datetime.now()